In [2]:
import pandas as pd
import folium
import os

df = pd.read_csv("./data/processed_bangalore_crime.csv")

df = df[['Latitude', 'Longitude', 'risk_score']]

# 🔥 Convert to numeric (VERY IMPORTANT)
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')

df = df.dropna()

# Remove invalid coordinates
df = df[
    (df['Latitude'].between(12.7, 13.2)) &
    (df['Longitude'].between(77.4, 77.8))
]

grid_size = 0.01

df['lat_grid'] = (df['Latitude'] // grid_size) * grid_size
df['lon_grid'] = (df['Longitude'] // grid_size) * grid_size

grid_risk = (
    df.groupby(['lat_grid', 'lon_grid'])['risk_score']
    .mean()
    .reset_index()
)

def risk_zone(score):
    if score <= 2:
        return 'Safe'
    elif score <= 4:
        return 'Moderate'
    else:
        return 'High Risk'

grid_risk['zone'] = grid_risk['risk_score'].apply(risk_zone)

m = folium.Map(location=[12.9716, 77.5946], zoom_start=12)

for _, row in grid_risk.iterrows():
    color = 'green' if row['zone'] == 'Safe' else \
            'orange' if row['zone'] == 'Moderate' else 'red'

    folium.Circle(
        location=[row['lat_grid'], row['lon_grid']],
        radius=400,
        color=color,
        fill=True,
        fill_opacity=0.6
    ).add_to(m)

os.makedirs("./output", exist_ok=True)

m.save("./output/bangalore_risk_map.html")

grid_risk[['lat_grid', 'lon_grid', 'zone', 'risk_score']].to_json(
    "./data/bangalore_risk_zones.json",
    orient='records'
)

print("✅ Bangalore risk zones created successfully.")

✅ Bangalore risk zones created successfully.
